In [ ]:
# # Read the contents of the original file with explicit encoding
# with open('course.txt', 'r', encoding='utf-8') as file:
#     lines = file.readlines()

# values_dicts = []
# keys = []
# for line in lines:
#     key, values = line.split(':',1)
#     key = key.strip()
#     # print(key)
#     values_dict = eval(values)
#     # print(values_dict)
#     values_dicts.append(values_dict)
#     keys.append(key)


# import pandas as pd
# df = pd.DataFrame(values_dicts, index = keys)
# df = df.rename_axis('course_id')
# df = df.reset_index()
# df.to_csv('courses.csv', index=False)

In [2]:
import pandas as pd
df = pd.read_csv("courses.csv")
df.head()

,course_id,course_title,training_organisation,uen,full_fee,no_of_respondents,course_attendee_count,course_objective,course_content,relevant_job_roles,entry_requirement,training_commitment,duration_hour,url,area_of_training
0,TGS-2023019838,Real Estate Salesperson (RES) Course,SEAA,T15SS0188B,$768.52,335.0,1053.0,The objectives of the RES Examination are to e...,The RES course prepares new entrants with Real...,NaN,Singapore Citizen & SPR 21 years old above,Full Time,54.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"
1,TGS-2020501266,Value Investing Programme (Interpretation of F...,MIND KINESIS INVESTMENTS PTE. LTD.,201202566W,$500.00,139.0,1103.0,Value Investing was founded and taught by Prof...,1) Basics of Securities Analysis\n2) Interpret...,NaN,"O Level, A Level, Diploma",Part Time,13.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"
2,TGS-2022016789,User Experience Design Bootcamp,VERTICAL INSTITUTE PTE. LTD.,201923625H,"$1,650.00",183.0,596.0,-Have a comprehensive understanding of UX desi...,Vertical Institute’s User Experience Design Bo...,NaN,This is a beginner-friendly program with no pr...,Part Time,28.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"
3,TGS-2023022106,Cybersecurity Bootcamp,VERTICAL INSTITUTE PTE. LTD.,201923625H,"$1,650.00",167.0,521.0,-Understand the basic of cyber-security and th...,Vertical Institute’s Cybersecurity Bootcamp la...,NaN,This is a beginner-friendly program with no pr...,Part Time,28.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"
4,TGS-2020000386,Graduate Certificate in Digital Finance Module...,SINGAPORE MANAGEMENT UNIVERSITY,200000267Z,"$1,600.00",102.0,600.0,"At the end of the 2-day module, participants w...",The module focuses on the fundamentals of the ...,NaN,NaN,NaN,17.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path = "courses.csv", encoding='utf8')
data = loader.load()


In [5]:
import transformers
import torch
from transformers import (
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  pipeline
)

from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain


MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'
##############################################
#TOKENIZER 
##############################################
model_config =transformers.AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


In [6]:
#####################################
#BitsAndBytes parameters
#####################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [7]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [8]:
print(torch.cuda.is_available())

True


In [9]:
################################################################
# Load pre-trained config
################################################################

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, 
    quantization_config=bnb_config,
    )

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print_number_of_trainable_model_parameters(model)

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


bin c:\Users\user\Desktop\AiSingapore24\.venv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.48s/it]
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap = 32
)

chunked_data = text_splitter.split_documents(data)

# load chunked data into the FAISS index
db = FAISS.from_documents(chunked_data,
                          HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

retriever = db.as_retriever()


In [14]:
# creating prompt template
prompt_template = """
### [INST] Role: Expert Course Consultant and Career Advisor
Instruction:
Given {question}, recommend suitable courses from the list to help them upskill or switch careers.

Do not recommend courses that the user already has relevant skills in.

Context:
{context}
Constraint:
Only recommend courses from the provided context.
Do not recommend courses to the user that they already have relevant skills in.
Examples:
Example 1:
User background: Software engineer with experience in Python programming.
User goal: Switch careers to data science.
Recommended course: Master of Science in Data Science (Course ID: TGS-2022014263, Fee: $36,500.00, URL: https://www.myskillsfuture.gov.sg/content/portal/en/training-exchange/course-directory/course-detail.html?courseReferenceNumber=TGS-2022014263)
Reasoning: This Master of Science in Data Science (MSDS) program is designed for students who want to gain a comprehensive understanding of data science. The program covers a wide range of topics, including data science thinking, machine learning, data systems, data preparation, data visualization, and social science. Students will also have the opportunity to complete a capstone project, which will allow them to apply their knowledge to a real-world problem.

This program is a great option for people who are switching from software engineering to data science because it provides a strong foundation in the essential concepts and skills needed to be successful in this field. The program also includes courses on social science and psychology, which can help data scientists to better understand the human impact of their work.

Here are some of the key benefits of this program for software engineers who want to switch to data science:

Covers a wide range of data science topics
Provides a strong foundation in essential concepts and skills
Includes courses on social science and psychology
Opportunity to complete a real-world capstone project


[/INST]

"""
prompt = PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | llm_chain
)

rag_chain.invoke("Hi, I am looking for a career switch from chemical engineering to artificial intelligence. What are the possible courses to take, I have beginner programming experience and have learnt basic python programming.")


c:\Users\user\Desktop\AiSingapore24\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': [Document(page_content='relevant_job_roles: \nentry_requirement: -A local polytechnic diploma/degree in science or engineering with a chemistry component (e.g. Life Sciences, Chemical Sciences, Chemical Engineering).', metadata={'source': 'courses.csv', 'row': 1822}),
  Document(page_content='relevant_job_roles: \nentry_requirement: -A local polytechnic diploma/degree in science or engineering with a chemistry component (e.g. Life Sciences, Chemical Sciences, Chemical Engineering).', metadata={'source': 'courses.csv', 'row': 1823}),
  Document(page_content='relevant_job_roles: \nentry_requirement: -A local polytechnic diploma/degree in science or engineering with a chemistry component (e.g. Life Sciences, Chemical Sciences, Chemical Engineering).', metadata={'source': 'courses.csv', 'row': 1824}),
  Document(page_content='relevant_job_roles: \nentry_requirement: -A local polytechnic diploma/degree in science or engineering with a chemistry component (e.g. Life Sciences, Che